In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install chardet
!pip install PyPDF2
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.3 MB/s eta 0:00:00


In [ ]:
import csv
import re
import pandas as pd
from PyPDF2 import PdfReader
from docx import Document
from datetime import datetime
import numpy as np

In [ ]:
%cd /content/drive/MyDrive/24-1NLP/CONF/data

/content/drive/MyDrive/24-1NLP/CONF/data


In [ ]:
import pandas as pd
from docx import Document

# Load the DOCX file
file_path = './friends_transcript.docx'
document = Document(file_path)

# Initialize lists to store scene, character, and dialogue information
scenes = []
characters = []
dialogues = []

current_scene = None

# Iterate over paragraphs in the document to extract information
for para in document.paragraphs:
    text = para.text.strip()

    # Identify scene changes
    if text.startswith("[Scene:"):
        current_scene = text
    elif ": " in text:
        # Split text into character and dialogue
        character, dialogue = text.split(": ", 1)
        scenes.append(current_scene)
        characters.append(character)
        dialogues.append(dialogue)

# Create a DataFrame from the extracted information
df = pd.DataFrame({
    'Scene': scenes,
    'Character': characters,
    'Dialogue': dialogues
})

# Replace '\xa0' with space
# df = df.applymap(lambda x: x.replace('\xa0', ' '))

# Save the DataFrame to a CSV file
output_csv_path = './friends_transcript.csv'
df.to_csv(output_csv_path, index=False)

print(f"CSV file saved to {output_csv_path}")


CSV file saved to ./friends_transcript.csv


In [ ]:
# 워드 파일 열기
# doc = Document('/content/drive/MyDrive/24-1NLP/friends_transcript_0605.docx')

### docx file -> chatgpt로 csv로 변환(발화자, 상황, 화면설명)
### csv에서 공백 이상하게 들어가 있는 경우 제외하기

f = pd.read_csv("./friends_transcript.csv")
f = f.iloc[:,[1,2]]
f = f.applymap(lambda x: x.replace('\xa0', ' ')) # # \xa0를 공백으로 바꾸기
# f.head()

In [ ]:
# Function to extract text from PDF
def extract_text_from_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# Function to parse the extracted text and extract dialogues
def parse_dialogues(text):
    dialogues = []
    lines = text.split("\n")#[10:] # 앞부분 제거해주고 lines 지정
    for line in lines:
        # print(line)
        if ":" in line:
            parts = line.split(":", 1)
            speaker = parts[0].strip()
            dialogue = parts[1].strip()
            if dialogue:
                dialogues.append((speaker, dialogue))

        elif (":" not in line) & (line[0] != '[') & (line[0] != '('):
            dialogues.pop()
            dialogue = (dialogue + f' {line}').strip()
            dialogues.append((speaker, dialogue))

    return dialogues

def make_dialogues(f):
    conversation_pairs = []
    for i in range(0, f.shape[0]-1):
        current_speaker = f.loc[i,'Character']
        current_line = re.sub(r"\[.*\]|\(.*\)|\{.*\}|-", "",f.loc[i,'Dialogue']).strip()
        next_speaker = f.loc[i+1,'Character']
        next_line = re.sub(r"\[.*\]|\(.*\)|\{.*\}|-", "",f.loc[i+1,'Dialogue']).strip()
        if (current_speaker.isalpha()) & (next_speaker.isalpha()): # 발화자/응답자 처리
            conversation_pairs.append((current_speaker, current_line, next_speaker, next_line))
    return conversation_pairs

# Function to create conversation pairs from dialogues
def create_conversation_pairs(dialogues):
    conversation_pairs = []
    for i in range(len(dialogues) - 1):
        current_speaker = dialogues[i][0]
        current_line = re.sub(r"\[.*\]|\(.*\)|\{.*\}|-", "",dialogues[i][1]).strip()  # re.sub("\(.*\)","",dialogues[i][1]).strip() r'\[.*\]|\(.*\)|\{.*\}|-'
        next_speaker = dialogues[i + 1][0]
        next_line = re.sub(r"\[.*\]|\(.*\)|\{.*\}|-", "",dialogues[i+1][1]).strip()
        if (current_speaker.isalpha()) & (next_speaker.isalpha()): # 발화자/응답자 처리
            conversation_pairs.append((current_speaker, current_line, next_speaker, next_line))
    return conversation_pairs

# Function to write conversation pairs to a CSV file
def write_to_csv(conversation_pairs, csv_path):
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Speaker", "Line", "NextSpeaker", "NextLine"])
        writer.writerows(conversation_pairs)


In [ ]:
# Paths to the input PDF file and output CSV file
# input_pdf_path = '/content/drive/MyDrive/24-1NLP/Friends Season01_EnglishScript.pdf'
output_csv_path = f"./friends_conversation_dataset_{datetime.now().date().strftime('%m%d')}.csv"

In [ ]:
# # Parse the extracted text to get dialogues
# dialogues = parse_dialogues(text)

# # Create conversation pairs from dialogues
# conversation_pairs = create_conversation_pairs(dialogues)

# Create conversation pairs from revised file
conversation_pairs = make_dialogues(f)

print(len(conversation_pairs))

6641


In [ ]:
df = pd.DataFrame(conversation_pairs, columns=["Speaker", "Line", "NextSpeaker", "NextLine"])

# 공백을 NaN으로 변환
df.replace('', np.nan, inplace=True)

# NaN이 포함된 행 삭제
df.dropna(inplace=True)

print(df.head())
print(len(df))

    Speaker                                               Line NextSpeaker  \
0    Monica  There's nothing to tell! He's just some guy I ...        Joey   
1      Joey  C'mon, you're going out with the guy! There's ...    Chandler   
2  Chandler  All right Joey, be nice.  So does he have a hu...      Phoebe   
3    Phoebe                           Wait, does he eat chalk?      Phoebe   
4    Phoebe  Just, 'cause, I don't want her to go through w...      Monica   

                                            NextLine  
0  C'mon, you're going out with the guy! There's ...  
1  All right Joey, be nice.  So does he have a hu...  
2                           Wait, does he eat chalk?  
3  Just, 'cause, I don't want her to go through w...  
4  Okay, everybody relax. This is not even a date...  
6498


In [ ]:
df = df[df['Line'].apply(len) >= 10].reset_index(drop = True)

In [ ]:
df.head(30)

,Speaker,Line,NextSpeaker,NextLine
0,Monica,There's nothing to tell! He's just some guy I ...,Joey,"C'mon, you're going out with the guy! There's ..."
1,Joey,"C'mon, you're going out with the guy! There's ...",Chandler,"All right Joey, be nice. So does he have a hu..."
2,Chandler,"All right Joey, be nice. So does he have a hu...",Phoebe,"Wait, does he eat chalk?"
3,Phoebe,"Wait, does he eat chalk?",Phoebe,"Just, 'cause, I don't want her to go through w..."
4,Phoebe,"Just, 'cause, I don't want her to go through w...",Monica,"Okay, everybody relax. This is not even a date..."
5,Monica,"Okay, everybody relax. This is not even a date...",Chandler,Sounds like a date to me.
6,Chandler,Sounds like a date to me.,Chandler,"Alright, so I'm back in high school, I'm stand..."
7,Chandler,"Alright, so I'm back in high school, I'm stand...",All,"Oh, yeah. Had that dream."
8,All,"Oh, yeah. Had that dream.",Chandler,"Then I look down, and I realize there's a phon..."
9,Chandler,"Then I look down, and I realize there's a phon...",Joey,Instead of...?


In [ ]:
# Write conversation pairs to a CSV file
df.to_csv(output_csv_path, index=False)

print(f"Conversation dataset created at {output_csv_path}")


Conversation dataset created at ./friends_conversation_dataset_0630.csv
